In [1]:
import time, sys, os
import rospy
from ros import rosbag
import roslib
roslib.load_manifest('sensor_msgs')
from sensor_msgs.msg import Image

import ImageFile

In [2]:
def GetFilesFromDir(dir):
    '''Generates a list of files from the directory'''
    print( "Searching directory %s" % dir )
    all = []
    left_files = []
    right_files = []
    if os.path.exists(dir):
        for path, names, files in os.walk(dir):
            for f in files:
                if os.path.splitext(f)[1] in ['.bmp', '.png', '.jpg']:
                    if 'left' in f or 'left' in path:
                        left_files.append( os.path.join( path, f ) )
                    elif 'right' in f or 'right' in path:
                        right_files.append( os.path.join( path, f ) )
                    all.append( os.path.join( path, f ) )
    return all, left_files, right_files

In [3]:
def CreateMonoBag(imgs, topic, bagname):
    '''Creates a bag file with camera images'''
    bag = rosbag.Bag(bagname, 'w')

    try:
        for i in range(len(imgs)):
#             print("Adding %s" % imgs[i])
            fp = open( imgs[i], "r" )
            p = ImageFile.Parser()

            while 1:
                s = fp.read(1024)
                if not s:
                    break
                p.feed(s)

            im = p.close()
            
            #TODO: change this to read time from filename string
            stamp_sting = os.path.splitext(os.path.basename(imgs[i]))[0]
            Stamp = rospy.Time.from_sec(float(stamp_sting))

            Img = Image()
            Img.header.stamp = Stamp
            Img.width = im.size[0]
            Img.height = im.size[1]
            Img.encoding = "mono8"
            Img.header.frame_id = "camera_rgb_optical_frame"
            Img_data = [pix for pix in im.getdata()]
            Img.data = Img_data

            bag.write(topic, Img, Stamp)
    finally:
        bag.close()

In [4]:
def CreateBag(imagedir, topic, bagname):
    '''Creates the actual bag file by successively adding images'''
    all_imgs, left_imgs, right_imgs = GetFilesFromDir(imagedir)
    if len(all_imgs) <= 0:
        print("No images found in %s" % args[0])
        exit()

    CreateMonoBag(all_imgs, topic, bagname)

In [5]:
bagfiles = ['new.optical4.ojjuice.bag']
path = '/home/ruffsl/Downloads/lol/'
imagedir = '/home/ruffsl/Downloads/lol/edge_images'
topic_name='/camera/rgb/image_edge'
for bagfile in bagfiles:
    CreateBag(imagedir, topic_name, path+bagfile)

Searching directory /home/ruffsl/Downloads/lol/edge_images


In [ ]:
# for bagfile in bagfiles:
#     !rosbag reindex '/home/ruffsl/Downloads/lol/optical4.ojjuice.bag'

`rosrun bagedit bagmerge.py -o edge.optical4.ojjuice.bag -i optical4.ojjuice.bag new.optical4.ojjuice.bag`

https://bitbucket.org/daniel_dube/bagedit/wiki/Home